In [1]:
import pandas as pd
import numpy as np

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection
from blimp.processing.segment_and_quantify import _get_channel_names

23-Oct-24 20:24:01 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
intensity_image_dir = Path("/srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/OME-TIFF-MIP")
label_image_dir = Path("/srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/SEGMENTATION")

intensity_image = AICSImage(intensity_image_dir / "Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")
intron_label_image = AICSImage(label_image_dir / "intron_in_nucleus_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")
nuclei_label_image = AICSImage(label_image_dir / "nuclei_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")
intron_nuclei_labels_label_image = AICSImage(label_image_dir / "nuclei_intron_masked_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff")

23-Oct-24 20:24:10 - bfio.bfio.BioReader - DEBUG    - Starting the backend...
23-Oct-24 20:24:10 - bfio.backends.PythonReader - DEBUG    - __init__(): Initializing _rdr (tifffile.TiffFile)...
23-Oct-24 20:24:10 - bfio.backends.PythonReader - DEBUG    - read_metadata(): Reading metadata...
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 256 ImageWidth @24 LONG @36 = 2304
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 257 ImageLength @44 LONG @56 = 2304
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 258 BitsPerSample @64 SHORT @76 = 16
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 259 Compression @84 SHORT @96 = ADOBE_DEFLATE
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 262 PhotometricInterpretation @104 SHORT @116 = MINISBLACK
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 270 ImageDescription @124 ASCII[843] @312 = <OME xmlns="http://www.open
23-Oct-24 20:24:10 - bfio.backends - DEBUG    - TiffTag 273 StripOffsets @144 LONG8[165] 

We want to modify the original quantify function to allow for the possibility that the labels do not match between the different channels in the image. We should therefore change the logic so that quantify_single_timepoint operates on only a single channel of the label image at a time. This will generate independent features tables for each object in the label image. However, we now also want to also be able to specify a "parent channel" which is where the true id can be found. 

In [4]:
from blimp.utils import concatenate_images
from typing import Union, Optional, List
import skimage.measure
from blimp.processing.segment_and_quantify import border_objects

def _measure_parent_object_label(
    label_image: AICSImage,
    measure_object_index: int,
    parent_object_index: int,
    timepoint: int = 0):

    label_array = label_image.get_image_data("YX", C=measure_object_index, T=timepoint, Z=0)
    parent_label_array = label_image.get_image_data("YX", C=parent_object_index, T=timepoint, Z=0)

    # mask the parent object array using the label array
    parent_object_array_masked = np.where(label_array > 0, parent_label_array, 0)

    parent_id = pd.DataFrame(
        skimage.measure.regionprops_table(
            label_image=label_array,
            intensity_image=parent_object_array_masked,
            properties=[
                "label",
                "intensity_min",
                "intensity_max",
            ],
            separator="_",
        )
    )

    # check that each object has a unique parent id
    # (i.e. the child object is fully contained within the parent object)
    columns_match = (parent_id['intensity_min'] == parent_id['intensity_max']).all()

    if not columns_match:
        raise ValueError

    parent_id['parent_label'] = np.floor(parent_id['intensity_max']).astype(label_image.dtype)
    parent_id['parent_label_name'] = label_image.channel_names[parent_object_index]
    parent_id = parent_id.drop(['intensity_min','intensity_max'], axis=1)

    return parent_id


def _quantify_single_timepoint(
    intensity_image: AICSImage,
    label_image: AICSImage,
    measure_object: Union[int, str],
    parent_object: Optional[Union[int, str]] = None,
    timepoint: int = 0,
    intensity_channels: Optional[Union[int, str, List[Union[int, str]]]] = None,
    calculate_textures: Optional[bool] = False,
    texture_channels: Optional[Union[int, str, List[Union[int, str]]]] = None,
    texture_scales: list = [1, 3],
) -> pd.DataFrame:
    """Quantify all channels in an image relative to a matching label image.
    Single time-point only. Single object only.

    Parameters
    ----------
    intensity_image
        intensity image (possibly 5D: "TCZYX")
    label_image
        label image (possibly 5D: "TCZYX")
    timepoint
        which timepoint should be quantified
    intensity_channels
        channels in ``intensity_image`` to be used for intensity calculations,
        can be provided as indices or names (see ``_get_channel_names()``)
    texture_channels
        channels in ``intensity_image`` to be used for texture calculations,
        can be provided as indices or names (see ``_get_channel_names()``)
    texture_scales
        length scales at which to calculate textures

    Returns
    -------
    pandas.DataFrame
        quantified data (n_rows = # objects, n_cols = # features)
    """

    features_list = []

    def intensity_sd(regionmask, intensity_image):
        return np.std(intensity_image[regionmask])

    def intensity_median(regionmask, intensity_image):
        return np.median(intensity_image[regionmask])

    # channels can be passed as names or indices, convert to names.
    intensity_channels_list = _get_channel_names(intensity_image, intensity_channels)
    
    measure_object = _get_channel_names(label_image, measure_object)[0]
    measure_object_index = label_image.channel_names.index(measure_object)
    if parent_object is not None:
        parent_object = _get_channel_names(label_image, parent_object)[0]
        parent_object_index = label_image.channel_names.index(parent_object)

    if calculate_textures:
        texture_channels_list = _get_channel_names(intensity_image, texture_channels)

    label_array = label_image.get_image_data("YX", C=measure_object_index, T=timepoint, Z=0)

    # Morphology features
    # -----------------------
    features = pd.DataFrame(
        skimage.measure.regionprops_table(
            label_array,
            properties=[
                "label",
                "centroid",
                "area",
                "area_convex",
                "axis_major_length",
                "axis_minor_length",
                "eccentricity",
                "extent",
                "feret_diameter_max",
                "solidity",
                "perimeter",
                "perimeter_crofton",
                "euler_number",
            ],
            separator="_",
        )
    ).rename(columns=lambda x: measure_object + "_" + x if x != "label" else x)
    features = features.merge(
        border_objects(label_array).rename(columns=lambda x: measure_object + "_" + x if x != "label" else x), on="label"
    )
    # Intensity features
    # ----------------------
    # iterate over selected channels
    for channel in intensity_channels_list:
        channel_index = intensity_image.channel_names.index(channel)
        intensity_array = intensity_image.get_image_data("YX", C=channel_index, T=timepoint, Z=0)

        intensity_features = pd.DataFrame(
            skimage.measure.regionprops_table(
                label_array,
                intensity_array,
                properties=["label", "intensity_mean", "intensity_max", "intensity_min"],
                extra_properties=(intensity_sd, intensity_median),
                separator="_",
            )
        ).rename(columns=lambda x: measure_object + "_" + x + "_" + channel if x != "label" else x)
        features = features.merge(intensity_features, on="label")

    # Texture features
    # ----------------------
    # iterate over selected channels
    if calculate_textures:
        for channel in texture_channels_list:
            channel_index = intensity_image.channel_names.index(channel)
            intensity_array = intensity_image.get_image_data("YX", C=channel_index, T=timepoint, Z=0)

            bboxes = mh.labeled.bbox(label_array)
            texture_features_list = [
                _calculate_texture_features_single_object(
                    intensity_array=intensity_array,
                    channel_name=channel,
                    object_name=measure_object,
                    bboxes=bboxes,
                    label=label,
                    scales=texture_scales,
                )
                for label in np.unique(label_array[label_array != 0])
            ]
            # collect data for all objects and merge with morphology/intensity features
            texture_features = pd.concat(texture_features_list, ignore_index=False)
            features = features.merge(texture_features, on="label")

    if parent_object is not None:
        parent_object_label = _measure_parent_object_label(label_image, measure_object_index, parent_object_index)
        features = features.merge(parent_object_label, on="label")

    # add timepoint information (note + 1 to match metadata)
    features[["TimepointID"]] = timepoint + 1
    return features



In [77]:
from blimp.utils import check_uniform_dimension_sizes

def make_channel_names_unique(image: AICSImage) -> AICSImage:
    if image.channel_names == None or image.channel_names == []:
        out = AICSImage(
            image.data,
            channel_names=[f"Channel_{index}" for index in range(image.dims.C)],
            physical_pixel_sizes=image.physical_pixel_sizes,
        )
        logger.warning(f"Channel names missing or empty, renaming to {out.channel_names}")
    elif len(set(image.channel_names)) != image.dims.C:
        out = AICSImage(
            image.data,
            channel_names=[f"{name}_{index}" for index, name in enumerate(image.channel_names)],
            physical_pixel_sizes=image.physical_pixel_sizes,
        )
        logger.warning(f"Channel names not unique, renaming to {out.channel_names}")
    else:
        out = image
        logger.info(f"Channel names unique: {out.channel_names}")
    return out


def aggregate_and_merge_dataframes(df_list: List[pd.DataFrame], parent_index: int):

    parent_df = df_list[parent_index]
    non_parent_dfs = [df for i, df in enumerate(df_list) if i != parent_index]

    # Initialize an empty list to store the aggregated dataframes
    aggregated_dfs = []
    for df in non_parent_dfs:
        # Select only numeric columns for aggregation, excluding 'label', 'timepoint', and 'parent_label'
        numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(['label', 'TimepointID', 'parent_label'])
        aggregations = ['mean', 'min', 'max', 'std', 'median']

        # Perform aggregation on numeric columns
        agg_df = df.groupby('parent_label').agg(
            {col: aggregations for col in numeric_cols}
        ).reset_index()  # Reset the index after grouping

        agg_df.columns = ['_'.join(col).strip() if col[0]!='parent_label' else col[0] for col in agg_df.columns]

        # prepend object_name to 'count'
        object_count = df.groupby('parent_label').size().to_frame('count').reset_index()
        agg_df = agg_df.merge(object_count, how='left', left_on='parent_label', right_on='parent_label')

        aggregated_dfs.append(agg_df)

    # Merge all aggregated dataframes with the parent dataframe on parent_label -> label
    for agg_df in aggregated_dfs:
        out = parent_df.merge(agg_df, how='left', left_on='label', right_on='parent_label')

    # Replace NaN with 0 in columns that end with 'count'
    out.loc[:, out.columns.str.endswith('count')] = out.loc[:, out.columns.str.endswith('count')].fillna(0).astype(int)

    return out


def quantify(
    intensity_image: AICSImage,
    label_image: AICSImage,
    measure_objects: Optional[Union[int, str]] = None,
    parent_object: Optional[Union[int, str]] = None,
    aggregate: Optional[bool] = False,
    timepoint: Optional[int] = None,
    intensity_channels: Optional[Union[int, str, List[Union[int, str]]]] = None,
    calculate_textures: Optional[bool] = False,
    texture_channels: Optional[Union[int, str, List[Union[int, str]]]] = None,
    texture_objects: Optional[Union[int, str, List[Union[int, str]]]] = None,
    texture_scales: list = [1, 3],
) -> Union[pd.DataFrame, List[pd.DataFrame]]:
    """Quantify all channels in an image relative to a matching segmentation
    label image.

    Parameters
    ----------
    intensity_image
        intensity image (possibly 5D: "TCZYX")
    label_image
        label image (possibly 5D: "TCZYX")
    timepoint
        which timepoint should be segmented (optional,
        default None will segment all time-points)

    Returns
    -------
    pandas.DataFrame
        quantified data (n_rows = # objects x # timepoints, n_cols = # features)
    """

    # Check inputs
    check_uniform_dimension_sizes([label_image, intensity_image], omit='C', check_dtype=False)
    make_channel_names_unique(label_image)
    make_channel_names_unique(intensity_image)
    
    if measure_objects is None:
        logger.info(f"``objects`` not specified. Measuring features for all objects.")
        measure_objects = label_image.channel_names
    else:
        for obj in measure_objects:
            if obj not in label_image.channel_names:
                raise ValueError(f"object {obj} not found in label image channel names")

    if parent_object is None:
        logger.info(f"``parent_object`` not specified. Data will be aggregated relative to channel 0.")
        parent_object = label_image.channel_names[0]
    else:
        if isinstance(parent_object, int):
            parent_object = label_image.channel_names[parent_object]
        elif isinstance(parent_object, str):
            if parent_object not in label_image.channel_names:
                raise ValueError(f"parent object {parent_object} not found in label image channel names")

    # desired behavior is
    # if aggregate is True, then all channels will be aggregated relative to the parent channel -> one dataframe
    # if aggregate is False, then all channels will be measured separately, with the parent_label returned as a separate feature -> list of dataframes
    # need to add a check that child objects are all contained within parent objects
    
    # for each object to measure, do the measurement
    features_list = []
    for obj_index, obj in enumerate(measure_objects):
        logger.debug(f"Quantifying object {obj}.")
        if timepoint is None:
            if intensity_image.dims.Z > 1:
                # 3D quantification
                features = pd.concat(
                    [
                        _quantify_single_timepoint_3D(
                            intensity_image=intensity_image,
                            label_image=label_image,
                            timepoint=t,
                            intensity_channels=intensity_channels,
                            intensity_objects=intensity_objects,
                            calculate_textures=calculate_textures,
                            texture_channels=texture_channels,
                            texture_objects=texture_objects,
                            texture_scales=texture_scales,
                        )
                        for t in range(intensity_image.dims[["T"]][0])
                    ]
                )
            else:
                # 2D quantification
                features = pd.concat(
                    [
                        _quantify_single_timepoint(
                            intensity_image=intensity_image,
                            label_image=label_image,
                            measure_object=obj,
                            parent_object=parent_object,
                            timepoint=t,
                            intensity_channels=intensity_channels,
                            calculate_textures=calculate_textures,
                            texture_channels=texture_channels,
                            texture_scales=texture_scales,
                        )
                        for t in range(intensity_image.dims[["T"]][0])
                    ]
                )
        else:
            if intensity_image.dims.Z > 1:
                # 3D quantification
                features = _quantify_single_timepoint_3D(intensity_image, label_image, timepoint)
            else:
                # 2D quantification
                features = _quantify_single_timepoint(intensity_image, label_image, timepoint)

        features_list.append(features)

    if aggregate and len(measure_objects) > 1:
        output = aggregate_and_merge_dataframes(
            features_list,
            parent_index=label_image.channel_names.index(parent_object))
    else:
        output = features_list

    return output



In [78]:
from blimp.utils import change_image_dtype
all_channels = concatenate_images([change_image_dtype(nuclei_label_image,np.int32),
                    intron_label_image],axis='C',order='append')

23-Oct-24 20:48:40 - blimp.utils - DEBUG    - Concatenating by appending, on axis C


In [79]:
all_channels.channel_names

['Nuclei', 'Intron']

In [80]:
all_channels_sub = AICSImage(all_channels.data[:,:,:,:500,:500],
                             channel_names=all_channels.channel_names)
intensity_sub = AICSImage(intensity_image.data[:,:3,:,:500,:500],
                             channel_names=intensity_image.channel_names[:3])

23-Oct-24 20:48:41 - fsspec.local - DEBUG    - open file: /srv/scratch/berrylab/z5459895/NikonSpinningDisk/240809/JOB/20240809_164338_571/intron_threshold_testing/SEGMENTATION/intron_in_nucleus_Well02_Channel647,405,561,405_Seq0000_0002.ome.tiff


In [81]:
df = _quantify_single_timepoint(label_image=all_channels_sub,intensity_image=intensity_sub,measure_object='Intron',parent_object='Nuclei')
df.columns

Index(['label', 'Intron_centroid_0', 'Intron_centroid_1', 'Intron_area',
       'Intron_area_convex', 'Intron_axis_major_length',
       'Intron_axis_minor_length', 'Intron_eccentricity', 'Intron_extent',
       'Intron_feret_diameter_max', 'Intron_solidity', 'Intron_perimeter',
       'Intron_perimeter_crofton', 'Intron_euler_number', 'Intron_is_border',
       'Intron_intensity_mean_647', 'Intron_intensity_max_647',
       'Intron_intensity_min_647', 'Intron_intensity_sd_647',
       'Intron_intensity_median_647', 'Intron_intensity_mean_405',
       'Intron_intensity_max_405', 'Intron_intensity_min_405',
       'Intron_intensity_sd_405', 'Intron_intensity_median_405',
       'Intron_intensity_mean_561', 'Intron_intensity_max_561',
       'Intron_intensity_min_561', 'Intron_intensity_sd_561',
       'Intron_intensity_median_561', 'parent_label', 'parent_label_name',
       'TimepointID'],
      dtype='object')

In [82]:
df

,label,Intron_centroid_0,Intron_centroid_1,Intron_area,Intron_area_convex,Intron_axis_major_length,Intron_axis_minor_length,Intron_eccentricity,Intron_extent,Intron_feret_diameter_max,...,Intron_intensity_sd_405,Intron_intensity_median_405,Intron_intensity_mean_561,Intron_intensity_max_561,Intron_intensity_min_561,Intron_intensity_sd_561,Intron_intensity_median_561,parent_label,parent_label_name,TimepointID
0,1,3.384615,342.384615,13.0,13.0,4.318667,3.679465,0.523557,0.812500,4.472136,...,4.945861,141.0,227.384615,306.0,191.0,33.724081,224.0,2,Nuclei,1
1,2,66.500000,387.500000,4.0,4.0,2.000000,2.000000,0.000000,1.000000,2.236068,...,6.837397,149.0,194.000000,212.0,166.0,18.425526,199.0,2,Nuclei,1
2,27,392.000000,136.000000,5.0,5.0,2.529822,2.529822,0.000000,0.555556,3.000000,...,13.059862,165.0,324.000000,377.0,256.0,42.759794,320.0,12,Nuclei,1
3,28,407.733333,391.666667,15.0,15.0,4.777954,3.981909,0.552682,0.750000,5.099020,...,7.863841,170.0,305.466667,371.0,207.0,44.226488,320.0,14,Nuclei,1
4,29,415.800000,390.600000,5.0,5.0,3.098387,1.788854,0.816497,0.833333,3.162278,...,7.838367,179.0,263.200000,283.0,240.0,15.065192,265.0,14,Nuclei,1


In [84]:
df = quantify(label_image=all_channels_sub,intensity_image=intensity_sub,parent_object='Nuclei', aggregate=True)


23-Oct-24 20:48:59 - root     - INFO     - Channel names unique: ['Nuclei', 'Intron']
23-Oct-24 20:48:59 - root     - INFO     - Channel names unique: ['647', '405', '561']
23-Oct-24 20:48:59 - root     - INFO     - ``objects`` not specified. Measuring features for all objects.
23-Oct-24 20:48:59 - root     - DEBUG    - Quantifying object Nuclei.
23-Oct-24 20:48:59 - root     - DEBUG    - Quantifying object Intron.
/scratch/pbs.5856829.kman.restech.unsw.edu.au/ipykernel_3179750/2272280014.py:54: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  out.loc[:, out.columns.str.endswith('count')] = out.loc[:, out.columns.str.endswith('count')].fillna(0).astype(int)


In [74]:
# Remove the parent dataframe from the list

#df_list = df
#parent_index=0

parent_df = df_list[parent_index]
non_parent_dfs = [df for i, df in enumerate(df_list) if i != parent_index]
# Initialize an empty list to store the aggregated dataframes
aggregated_dfs = []
for df in non_parent_dfs:
    # Select only numeric columns for aggregation, excluding 'label', 'timepoint', and 'parent_label'
    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(['label', 'TimepointID', 'parent_label'])
    aggregations = ['mean', 'min', 'max', 'std', 'median']

    # Perform aggregation on numeric columns
    agg_df = df.groupby('parent_label').agg(
        {col: aggregations for col in numeric_cols}
    ).reset_index()  # Reset the index after grouping

    agg_df.columns = ['_'.join(col).strip() if col[0]!='parent_label' else col[0] for col in agg_df.columns]

    # prepend object_name to 'count'
    object_count = df.groupby('parent_label').size().to_frame('count').reset_index()
    agg_df = agg_df.merge(object_count, how='left', left_on='parent_label', right_on='parent_label')

    aggregated_dfs.append(agg_df)

# Merge all aggregated dataframes with the parent dataframe on parent_label -> label
for agg_df in aggregated_dfs:
    out = parent_df.merge(agg_df, how='left', left_on='label', right_on='parent_label')

# Replace NaN with 0 in columns that end with 'count'
out.loc[:, out.columns.str.endswith('count')] = out.loc[:, out.columns.str.endswith('count')].fillna(0).astype(int)


/scratch/pbs.5856829.kman.restech.unsw.edu.au/ipykernel_3179750/2942443715.py:33: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  out.loc[:, out.columns.str.endswith('count')] = out.loc[:, out.columns.str.endswith('count')].fillna(0).astype(int)


In [85]:
out

,label,Nuclei_centroid_0,Nuclei_centroid_1,Nuclei_area,Nuclei_area_convex,Nuclei_axis_major_length,Nuclei_axis_minor_length,Nuclei_eccentricity,Nuclei_extent,Nuclei_feret_diameter_max,...,Intron_perimeter_crofton_min,Intron_perimeter_crofton_max,Intron_perimeter_crofton_std,Intron_perimeter_crofton_median,Intron_solidity_mean,Intron_solidity_min,Intron_solidity_max,Intron_solidity_std,Intron_solidity_median,count
0,1,46.598261,68.438765,13342.0,13450.0,157.583331,111.863184,0.704336,0.852796,166.832251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,61.898141,402.645047,22541.0,22849.0,196.232146,154.264657,0.618057,0.804231,201.816749,...,6.473755,12.392149,4.184937,9.432952,1.0,1.0,1.0,0.0,1.0,2
2,8,112.057909,490.093834,1865.0,1927.0,106.614048,24.972990,0.972179,0.616529,121.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,12,333.153643,169.617681,21537.0,21894.0,201.772882,136.712177,0.735472,0.720470,199.832430,...,8.044551,8.044551,NaN,8.044551,1.0,1.0,1.0,NaN,1.0,1
4,14,356.159092,419.458745,30542.0,30716.0,214.254836,187.657227,0.482565,0.818601,230.384461,...,7.814513,13.732908,4.184937,10.773711,1.0,1.0,1.0,0.0,1.0,2
5,18,360.769542,9.633423,2226.0,2301.0,120.599474,24.559985,0.979044,0.754065,123.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,20,459.086389,271.678739,17699.0,17831.0,236.049698,102.954927,0.899870,0.719472,246.203168,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [30]:
# Perform aggregation on numeric columns

agg_df.columns = ['_'.join(col).strip() for col in agg_df.columns]

In [41]:
object_count

,parent_label,count
0,2,2
1,12,1
2,14,2
